<a href="https://colab.research.google.com/github/bingsen-wang/PowerElectronics/blob/main/DualActiveBridge_continuousPhaseShift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1><b>Dual Active Bridge - Continuous Phase Shift</h1></b>

Dr. Bingsen Wang

6/11/2023
</center>

#Python Code
The Python code illustrates the basic waveforms of a dual active bridge (DAB) with continuous phase-angle shifts.

In [ ]:
from sys import int_info
import matplotlib
from os import linesep
import numpy as np
from numpy import linspace,cos,pi,fft
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgba
from matplotlib.path import Path
import matplotlib.patches as mp
from matplotlib.collections import PatchCollection
from matplotlib import animation,rc,transforms
rc('animation', html='jshtml')
plt.rcParams.update({"font.family" : "serif","mathtext.fontset" : "cm"})

#square waveform
def h(t,f,phi):
  return np.heaviside(cos(2*pi*f*t+phi),0)

#indcutor currents
def il(vl,L,f):
  "R,L: values of the RL load; f: fundamental frequency"
  Iload_fft=[]
  Vload_fft = fft.rfft(vl)
  for k in range(np.shape(Vload_fft)[-1]):
    if k==0:
      Iload_fft.append(0)
    else:
      Iload_fft.append(Vload_fft[k]/(1j*k*2*pi*f*L))
  iload = fft.irfft(Iload_fft)
  return iload

#draw MOSFET symbol including body diode
def drawMOSFET(ax,x0,y0,h,theta,lw,color):
  codes=[Path.MOVETO,Path.LINETO,Path.LINETO]
  codes1=[Path.MOVETO,Path.LINETO,Path.LINETO,Path.CLOSEPOLY]
  p1=mp.PathPatch(Path([(x0,y0),(x0,y0+h*.5),(x0-h*.15,y0+h*.5),
                       (x0-h*.15,y0+h*.7),(x0,y0+h*.7),(x0,y0+h),
                       (x0-h*.225,y0+h*.7),(x0-h*.225,y0+h*.3),(x0-h*.35,y0+h*.3),
                       (x0,y0+h*.8),(x0+h*.15,y0+h*.8),(x0+h*.15,y0+h*.575),
                       (x0,y0+h*.2),(x0+h*.15,y0+h*.2),(x0+h*.15,y0+h*.425),
                       ], codes+codes+codes+codes+codes))
  p2=mp.PathPatch(Path([(x0-h*.15,y0+h*.3),(x0,y0+h*.3),
                       (x0-h*.15,y0+h*.775),(x0-h*.15,y0+h*.625),
                       (x0-h*.15,y0+h*.575),(x0-h*.15,y0+h*.425),
                       (x0-h*.15,y0+h*.375),(x0-h*.15,y0+h*.225),
                       (x0+h*.075,y0+h*.575),(x0+h*.225,y0+h*.575)],
                       codes[0:2]+codes[0:2]+codes[0:2]+codes[0:2]+codes[0:2]))
  p3=mp.PathPatch(Path([(x0-h*.075,y0+h*.535),(x0-h*.15,y0+h*.5),(x0-h*.075,y0+h*.465),(x0-h*.075,y0+h*.535),
                       (x0+h*.15,y0+h*.575),(x0+h*.075,y0+h*.425),(x0+h*.225,y0+h*.425),(x0+h*.15,y0+h*.575)
                      ],codes1+codes1)) #two triangles
  rot=transforms.Affine2D().rotate_deg_around(x0,y0,theta)+ax.transData
  ax.add_collection(PatchCollection([p1,p2,p3],ec=color,fc=['none','none',color],lw=lw,transform=rot))
  return

#draw inductor symbol
def drawInductor(ax,x0,y0,n,h,theta,lw,color):
  """n: number of coils;  h: height or width; lw: line weight
  theta: orientation 0 for horizontal 90 for vertical """
  w= h/(0.7*n+0.3)#width of larger coil
  plg, psm = ([],[]) #the half coil with larger curvature
  codes=[Path.MOVETO,Path.CURVE4,Path.CURVE4,Path.CURVE4]
  verts=[(x0,y0),(x0,y0+0.4*h),(x0+w,y0+0.4*h),(x0+w,y0)]
  for k in range(n):
    if k==0:
      plg.append(mp.PathPatch(Path(verts,codes)))
    else:
      p = Path([(x0 + 0.7*k*w,y0),(x0 + 0.7*k*w,y0+0.4*h),
                (x0+(0.7*k+1)*w,y0+0.4*h),(x0+(0.7*k+1)*w,y0)], codes)
      plg.append(mp.PathPatch(p))
      p = Path([(x0 + 0.7*k*w,y0),(x0 + 0.7*k*w,y0-0.2*h),
                (x0+(0.7*k+0.3)*w,y0-0.2*h),(x0+(0.7*k+0.3)*w,y0)], codes)
      psm.append(mp.PathPatch(p))
  rot=transforms.Affine2D().rotate_deg_around(x0,y0,theta)+ax.transData
  pc=PatchCollection(np.concatenate((plg,psm)),ec=color,fc='none',lw=lw,transform=rot,capstyle='round')
  ax.add_collection(pc)
  return

#draw capcitor symbol
def drawCapacitor(ax,x0,y0,h,theta,lw,color):
  """ h: height or width;   lw: line weight
  theta: orientation 0 for horizontal 90 for vertical """
  dx1,dx2,dy = (0.425*h, 0.575*h, 0.25*h)
  p = Path([(x0,y0),(x0+dx1,y0),(x0+dx1,y0+dy),(x0+dx1,y0-dy),
            (x0+dx2,y0+dy),(x0+dx2,y0-dy),(x0+dx2,y0),(x0+h,y0)],
           [Path.MOVETO,Path.LINETO, Path.MOVETO,Path.LINETO,
            Path.MOVETO, Path.LINETO,Path.MOVETO,Path.LINETO])
  rot=transforms.Affine2D().rotate_deg_around(x0,y0,theta)+ax.transData
  args = dict({'lw':lw,'capstyle':'round'})
  ax.add_patch(mp.PathPatch(p,transform=rot,fc='none',ec=color,**args))
  return

#draw the +/- voltage polarity with variable distance/color
def drawVoltSign(ax,x0,y0,h,theta,lw,color):
  """
  x0,y0: center between "+" and "-"
  h: height or width
  theta: orientation 0 for horizontal 90 for vertical
  lw: line weight
  """
  p = Path([(x0-0.5*h,y0-0.1*h),(x0-0.5*h,y0+0.1*h),(x0-0.6*h,y0),(x0-0.4*h,y0),
            (x0+0.5*h, y0-0.08*h),(x0+0.5*h, y0+0.08*h)],
           [Path.MOVETO,Path.LINETO,Path.MOVETO,Path.LINETO,
            Path.MOVETO,Path.LINETO])
  rot=transforms.Affine2D().rotate_deg_around(x0,y0,theta)+ax.transData
  ax.add_patch(mp.PathPatch(p,ec=color,fc='none',lw=lw,transform=rot,capstyle='round'))
  return

#draw the dual active bridge without switches, non-animated part
def drawDABna(ax,lw):
  cls=['r','b'] #use different colors for bridge1 &2
  codes=[Path.MOVETO,Path.LINETO]
  for k in range(2):
    ax.add_patch(mp.PathPatch(Path([(-4,2-4*k),(-1.25,2-4*k)],codes),lw=lw)) #+bus
    ax.add_patch(mp.PathPatch(Path([(4,2-4*k), ( 1.25,2-4*k)],codes),lw=lw)) #-bus
    ax.add_patch(mp.PathPatch(Path([(-2.25+3.5*k,1.1), ( -2.25+3.5*k,-1.1),
                                    (-1.25+3.5*k,1.1), ( -1.25+3.5*k,-1.1)],
                                   codes+codes),lw=lw))
    ax.add_patch(mp.PathPatch(Path([(3.8*(2*k-1),2),(3.8*(2*k-1),0.5),
                                    (3.8*(2*k-1),-2),(3.8*(2*k-1),-0.5)
                                    ],codes+codes),lw=lw)) #capacitor lines
    drawCapacitor(ax,3.8*(2*k-1),-.5,1,90,lw,'k')
  ax.add_patch(mp.PathPatch(Path([(-2.25,.75),(-.5,.75),(.25,.75),(1.25,.75)],
                                 codes+codes),lw=lw))
  ax.add_patch(mp.PathPatch(Path([(-1.25,-.75),(2.25,-.75)],codes),lw=lw))
  drawInductor(ax,-0.5,.75,5,.75,0,lw,'k')
  drawVoltSign(ax,-3.5,0,1,-90,lw,cls[0])
  drawVoltSign(ax,3.5,0,1,-90,lw,cls[1])
  ax.text(-3.5,0,'$V_{dc1}$',size=24,va='center',color=cls[0])
  ax.text(3.5,0,'$V_{dc2}$',size=24,va='center',ha='right',color=cls[1])
  ax.text(-1,0,'$v_1$',size=24,va='center',ha='center',color=cls[0])
  ax.text(1,0,'$v_2$',size=24,va='center',ha='center',color=cls[1])
  ax.text(.75,.8,'$i_L$',size=24,va='bottom',ha='center',color='c')
  ax.text(-.125,1,'$v_L$',size=24,va='bottom',ha='center',color='g')
  ax.text(-3,1.95,'$i_{dc1}$',size=24,va='top',ha='center',color=cls[0])
  ax.text(3,1.95,'$i_{dc2}$',size=24,va='top',ha='center',color=cls[1])
  return

#draw the animated part of the VSI
def drawDABanim(ax,h1,h2,v1,v2,vL,iL,idc1,idc2):
  h=[h1,h2]
  cls=['r','b'] #use different colors for bridge1 &2
  for k in range(2):
    drawMOSFET(ax,-2.25+k*3.5,1.1,.9,0,2, cls[k])
    drawMOSFET(ax,-1.25+k*3.5,1.1,.9,0,2, cls[k])
    drawMOSFET(ax,-2.25+k*3.5,-2,.9,0,2, cls[k])
    drawMOSFET(ax,-1.25+k*3.5,-2,.9,0,2, cls[k])
  drawVoltSign(ax,-1,0,v1*1.25,-90,3*v1,cls[0]) # for v1
  drawVoltSign(ax,1,0,v2*1.25,-90,3*v2,cls[1]) # for v2
  drawVoltSign(ax,-.125,1.1,vL,0,3*v2,'g') # for vL
  ax.add_patch(mp.Arrow(.75 - 0.5*iL,.75,iL,0,width=0.2,color='c')) #iL dir
  ax.add_patch(mp.Arrow(-3 - 0.5*idc1,2,idc1,0,width=0.2,color=cls[0])) #idc1 dir
  ax.add_patch(mp.Arrow(3 - 0.5*idc2,2,idc2,0,width=0.2,color=cls[1])) #idc2 dir
  return

#parameters
Vdc1=1
Vdc2=0.7
f=1
L=0.3

Nfpp = 120
Nf = Nfpp
phi2_lst=linspace(-pi/2,pi/2,Nf)
t=linspace(0,2/f,512)
phi1 = 0
ht1a = h(t,f,phi1) #switching function for bridge 1 leg-a
ht1b = h(t,f,phi1+pi) #switching function for bridge 1 leg-a
v1t = (ht1a-ht1b)*Vdc1

fig = plt.figure(figsize=(9,16))
fig.tight_layout()
ax_frame = [[[0,0.7 , 1, 0.3],[-4.5,4.5],[-2.4,2.4]], #diagram
            [[0,0.7 , 1, 0.3],[-4.5,4.5],[-2.4,2.4]], #diagram
            [[0,.52, 1, .18],[-.02,1.1],[-2.1,2.1]], #v1,v2
            [[0,.34, 1, .18],[-.02,1.1],[-2.1,2.1]], #vL,iL
            [[0,.16, 1, .18],[-.02,1.1],[-2.1,2.1]], #idc1,idc2
            [[0,0, 1, .16],[-.02,1.1],[-1.1,1.1]], #p1,p2,P12_avg
            ]# [pos-boundary, xlim, ylim] for subplots
ax_lbl=[['$v_1$', '$v_2$', '$t$'],
        ['$v_L$', '$i_L$', '$t$'],
        ['$i_{dc1}$','$i_{dc2}$', '$t$'],
        ['$p_1$','$p_2$','$P_{1,2\_avg}$', '$t$'],
        ] #variables for yx axes of each subplot
clst=['r','b','g','c','r','b','r','b','m'] #colors of lines
ax_lst=[] #axis list or the list of subplots
lines = [] #array containing all the line segments
for k in range(len(ax_frame)):
  # xn,xm,yn,ym=np.concatenate((ax_frame[k][1],ax_frame[k][2]))
  xn,xm,yn,ym = ax_frame[k][1]+ax_frame[k][2]
  ax=fig.add_axes(ax_frame[k][0],xlim=[xn,xm], ylim=[yn,ym],fc='none') #no fc
  ax.axis('off') #turn off axis frames
  ax_lst.append(ax)
  if k>1:
    ax.annotate("", (xm, 0), (xn, 0),arrowprops={'arrowstyle':"->"}) #x-axis
    ax.annotate("", (0,ym), (0,yn),arrowprops={'arrowstyle':"->"}) #y-axis
    for kk in range(len(ax_lbl[k-2])-1): #based on variable list
      lines.append(ax.plot([], [], color=clst[2*(k-2)+kk], lw=2)[0]) #lines to plot
      ax.text(xm-.02,0,ax_lbl[k-2][-1],size=24,va='top',ha='right') #x-axis label
      ax.text(0.01+kk*.07,ym,ax_lbl[k-2][kk],size=24,va='center',color=clst[2*(k-2)+kk]) #y label
txt_angleShift = ax_lst[2].text(0.5,ax_frame[2][2][1],'',va='top',ha='center',size=22)
# animation function. This is called sequentially
def animate(i):
  phi2=phi2_lst[i]
  txt_angleShift.set_text('Phase Shift = '+str(round(phi2/pi*180,1))+' degrees')
  ht2a = h(t,f,phi2) #switching function for bridge 2 leg-a
  ht2b = h(t,f,phi2+pi) #switching function for bridge 2 leg-b
  v2t = (ht2a-ht2b)*Vdc2
  vLt = v1t - v2t
  iLt = il(vLt,L,f/2)
  idc1t = ht1a*iLt - ht1b*iLt
  idc2t = ht2a*iLt - ht2b*iLt
  p1t = v1t*iLt
  p2t = v2t*iLt
  Pavg = np.average(p1t)
  lines[0].set_data(t/2,v1t)
  lines[1].set_data(t/2,v2t)
  lines[2].set_data(t/2,vLt)
  lines[3].set_data(t/2,iLt)
  lines[4].set_data(t/2,idc1t)
  lines[5].set_data(t/2,idc2t)
  lines[6].set_data(t/2,p1t)
  lines[7].set_data(t/2,p2t)
  lines[8].set_data(t/2,t*0+Pavg)
  return

drawDABna(ax_lst[0],2)
drawDABanim(ax_lst[1],1,1,1,1,1,.7,.7,.7)
anim = animation.FuncAnimation(fig, animate, frames=Nf, interval=50)
# anim #uncomment to generate animation in the output area
# to save the animation, uncomment the following three lines
fn = r"DualActiveBridge_continuousPhaseShift.mp4"
writervideo = animation.FFMpegWriter(fps=6)
anim.save(fn, writer=writervideo,dpi = 120)

#debug
# drawVSIanim(ax1,mht[:][1],iact[:][1],iht[:][1],idct[1])
# lines[4].set_data(t/2,idc1t)
# lines[5].set_data(t/2,idc2t)
# print(np.average(p1t))
# print(np.average(p2t))